# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [ ]:
 # Basic imports.
import os
import time
import numpy as np
import torch

from torch import nn
from torch import optim

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

In [ ]:
# Setting predefined arguments.
args = {
    'epoch_num': 10,      # Number of epochs.
    'lr': 0.0005,         # Learning rate.
    'weight_decay': 1e-5, # L2 penalty.
    'num_workers': 3,     # Number of workers on data loader.
    'batch_size': 50,     # Mini-batch size.
    'print_freq': 1,      # Printing frequency.
    'noise_std': 0.05,    # Artificial gaussian noise standard deviation.
    'noise_prob': 0.05,   # Artificial salt-and-pepper noise probability.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

# Carregando o FashionMNIST

In [ ]:
# Root directory for the dataset (to be downloaded).
root = './'

# Transformations over the dataset.
data_transforms = transforms.Compose([
    transforms.ToTensor()
])

# Setting datasets and dataloaders.
train_set = datasets.FashionMNIST(root,
                                  train=True,
                                  download=True,
                                  transform=data_transforms)
test_set = datasets.FashionMNIST(root,
                                 train=False,
                                 download=False,
                                 transform=data_transforms)

# Setting dataloaders.
train_loader = DataLoader(train_set,
                          args['batch_size'],
                          num_workers=args['num_workers'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                         args['batch_size'],
                         num_workers=args['num_workers'],
                         shuffle=False)

# Printing training and testing dataset sizes.
print('Size of training set: ' + str(len(train_set)) + ' samples')
print('Size of test set: ' + str(len(test_set)) + ' samples')

# Denoising AutoEncoder

Tanto AEs Lineares quanto Convolucionais podem ser adaptados para tarefas diferentes da de redução de dimensionalidade e reconstrução de imagens. Ao adicionar ruído artificial às imagens de entrada antes de passá-las para a forward, por exemplo, é possível treinar um AE para aprender a ignorar esse ruído e reconstruir as imagens originais, efetivamente realizando uma filtragem de ruído treinável. Em suma, é isso que um Denoising AutoEncoder (DAE) faz, como mostra o esquema abaixo.

![Denoising AE](https://www.dropbox.com/s/mekbapirkdvwkhx/Denoising_AE.png?dl=1)

É perceptível que ainda é preciso ter amostras limpas de ruído para treinar esse tipo de método, tornando-o limitado em alguns cenários. Porém, utilizando o conhecimento que se tem sobre as distribuições de ruídos em diferentes tipos de imagens (i.e. [gaussiano](https://en.wikipedia.org/wiki/Additive_white_Gaussian_noise), [salt-and-pepper](https://en.wikipedia.org/wiki/Salt-and-pepper_noise), [ruído quântico](https://www.sciencedirect.com/topics/chemistry/quantum-noise), etc) e realizando algumas suposições sobre a natureza das imagens, é possível construir com pequenas alterações no AE tradicional um removedor de ruído específico para os dados de um certo domínio.

O ruído salt-and-pepper é determinado pixel-a-pixel por uma distribuição de Bernoulli. Ou seja, dados $M$ pixels ${A_{0}, A_{1}, ..., A_{M-1}}$ de uma imagem $A$ e uma probabilidade $p$, cada pixel $A_{i}$ possui uma probabilidade $\frac{p}{2}$ de ser setado para 0 e uma probabilidade $\frac{p}{2}$ de ser setado para 1. No caso do ruído aditivo gaussiano, valores de uma imagem de ruído $B$ (tal que $B_{i} \sim N(0, std)$) amostrada aleatoriamente são somados a todos os pixels da imagem $A$, resultando numa imagem $C = A + B$.

Métodos mais modernos (i.e. [Noise2Noise](https://arxiv.org/pdf/1803.04189.pdf)) conseguem convergir sem a necessidade de dados não afetados pelos ruídos, tornando-os altamente aplicáveis em áreas como [imagens biomédicas](http://www.sprawls.org/ppmi2/NOISE/) ou [sensoriamento remoto](https://en.wikipedia.org/wiki/Synthetic-aperture_radar), as quais normalmente não possuem amostras "limpas" para treinamento de um DAE.

# Atividade Prática: modificando o AE Convolucional para remoção de ruído

1.   Implemente uma função que adicione um ruído do tipo salt-and-pepper em todas as imagens de um batch. Essa função deve receber um tensor de entrada e uma probabilidade $p$ de que cada pixel seja setado para 0 ou para 1, que são os valores mínimos das imagens. Dica: função [*torch.rand_like()*](https://pytorch.org/docs/stable/torch.html#torch.rand_like);
2.   Implemente uma função que adicione um ruído aditivo gaussiano em todas as imagens de um batch. Essa função deve receber um tensor de entrada e um desvio padrão $std$ da distribuição normal que será adicionada a cada pixel. Dica: função [*torch.randn_like()*](https://pytorch.org/docs/stable/torch.html#torch.randn_like);
3.   Modifique a arquitetura do AE Convolucional da aula passada para quadruplicar a quantidade de canais em cada camada convolucional do Encoder e do Decoder. Como nossa preocupação não é mais gerar uma representação compacta dos dados e sim remover ruído, não precisamos mais ser tão econômicos com os canais das convoluções;
4.   Modifique as funções *train()* e *test()* para adicionar ruído usando uma das duas funções a cima na variável *inps* antes de passá-la para a entrada do AE Convolucional;
5.   Varie os valores de args\['noise_std'\] e args\['noise_prob'\] para ver até qual nível de ruído os AEs conseguem reconstruir fidedignamente as imagens originais.


In [ ]:
# TO DO: Adding Salt-and-Pepper with probability p.
def add_salt_and_pepper_noise(tensor, p=0.05):
    # ...
    
    return noisy_tensor

In [ ]:
# TO DO: Inserting Additive Gaussian Noise with standard deviation std.
def add_additive_gaussian_noise(tensor, std=0.1):
    # ...
    
    return noisy_tensor